In [ ]:
!pip install torchxrayvision
!pip install timm
!pip install torchmetrics
!pip install einops

In [ ]:
#!unzip OCT2017.zip

In [ ]:
#import glob
#len(glob.glob("OCT2017/train/DME/*"))

In [ ]:
#@title Imports

import numpy as np
import torch
import time
import math
import os
import cv2
import timm
from timm.models.vision_transformer import VisionTransformer as timm_ViT
import torch
from torch import Tensor
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.parameter import Parameter
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from torchvision import datasets, transforms
from torchvision.transforms import InterpolationMode
from einops import rearrange
from safetensors import safe_open
from safetensors.torch import save_file
import torchxrayvision as xrv
from pathlib import Path
import pandas as pd
from sklearn.model_selection import train_test_split
from google.colab import drive

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

In [ ]:
from google.colab import drive

drive.mount('/content/drive/')
!unzip /content/drive/MyDrive/ATDL5/OCT2017.zip -d /content/
#!rm -r OCT2017/train/sample_data/

In [ ]:
import glob
len(glob.glob("/content/OCT2017/train/CNV/*"))
#len(glob.glob("/content/drive/MyDrive/ATDL5/OCT2017/train/TESTCNV/CNV/*"))

In [ ]:
!cp /content/drive/MyDrive/ATDL5/trainoct2.csv /content/trainoct2.csv


In [ ]:

#%cd drive/MyDrive/ATDL5
timse = time.time()
# #!mkdir OCT2017/train
# !cp -r /content/drive/MyDrive/ATDL5/OCT2017/train/CNV /content/OCT2017/train/
# !cp -r /content/drive/MyDrive/ATDL5/OCT2017/train/NORMAL /content/OCT2017/train/
# !cp -r /content/drive/MyDrive/ATDL5/OCT2017/train/DME /content/OCT2017/train/
# !cp -r /content/drive/MyDrive/ATDL5/OCT2017/train/CNV /content/OCT2017/train/


timse = time.time() - timse
print(timse)

147.93058276176453

In [ ]:
#@title LoraLayer
class _LoRALayer(nn.Module):
    def __init__(self, w: nn.Module, w_a_q: nn.Module, w_b_q: nn.Module,
                                 w_a_v: nn.Module, w_b_v: nn.Module, r: int, alpha: int, dim:int):
        super().__init__()
        self.w = w

        self.w_a_q = w_a_q
        self.w_b_q = w_b_q

        self.w_a_v = w_a_v
        self.w_b_v = w_b_v

        self.r = r
        self.alpha = alpha
        self.dim = dim

    def forward(self, x):
        val = self.w(x)
        val[:,:,:self.dim] += (self.alpha // self.r) * self.w_b_q(self.w_a_q(x))
        val[:,:,self.dim*2:] += (self.alpha // self.r) * self.w_b_v(self.w_a_v(x))
        return val
class LoRA_ViT(nn.Module):
    def __init__(self, vit_model, r: int, alpha: int, num_classes: int = 0, lora_layer=None):
        super(LoRA_ViT, self).__init__()

        base_vit_dim = vit_model.blocks[0].attn.qkv.in_features
        dim = base_vit_dim
        if lora_layer:
            self.lora_layer = lora_layer
        else:
            self.lora_layer = list(range(len(vit_model.blocks)))

        self.w_As = nn.ModuleList()
        self.w_Bs = nn.ModuleList()


        for param in vit_model.parameters():
            param.requires_grad = False

        for t_layer_i, blk in enumerate(vit_model.blocks):

            if t_layer_i not in self.lora_layer:
                continue
            #w_q_linear = blk.attn.proj_q
            #w_v_linear = blk.attn.proj_v
            w_qkv_linear = blk.attn.qkv

            w_a_linear_q = nn.Linear(dim, r, bias=False)
            w_b_linear_q = nn.Linear(r, dim, bias=False)


            w_a_linear_v = nn.Linear(dim, r, bias=False)
            w_b_linear_v = nn.Linear(r, dim, bias=False)


            self.w_As.append(w_a_linear_q)
            self.w_Bs.append(w_b_linear_q)
            self.w_As.append(w_a_linear_v)
            self.w_Bs.append(w_b_linear_v)

            blk.attn.qkv = _LoRALayer(w_qkv_linear, w_a_linear_q, w_b_linear_q, w_a_linear_v, w_b_linear_v,r, alpha, dim)

        self.reset_parameters()
        self.lora_vit = vit_model
    def save_fc_parameters(self, filename: str) -> None:
        assert filename.endswith(".safetensors")
        _in = self.lora_vit.fc.in_features
        _out = self.lora_vit.fc.out_features
        fc_tensors = {f"fc_{_in}in_{_out}out": self.lora_vit.fc.weight}
        save_file(fc_tensors, filename)
    def reset_parameters(self) -> None:
        for w_A in self.w_As:
            nn.init.kaiming_uniform_(w_A.weight, a=math.sqrt(5))
        for w_B in self.w_Bs:
            nn.init.zeros_(w_B.weight)
    def forward(self, x: Tensor) -> Tensor:
        return self.lora_vit(x)
    def get_intermediate_layers(self, x: torch.Tensor, n,  reshape = False, return_class_token= False,norm=True):
        return self.lora_vit.get_intermediate_layers(x, n, reshape, return_class_token, norm)


In [ ]:
#@title Transformer

transform_toTensor = transforms.Lambda(lambda x: torch.tensor(np.array(x), dtype=torch.uint8).unsqueeze(0))
#to_rgb = transforms.Lambda(lambda x: np.stack([x] * 3, axis=-1))

transform_train = transforms.Compose([
  #  to_rgb,
    transforms.ToTensor(),
    transforms.RandomResizedCrop(size = (224,224), scale=(0.2, 1)),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.RandomHorizontalFlip(p=0.5),
    #transforms.Normalize(mean=[0.4823, 0.4823, 0.4823],std = [0.2232, 0.2232, 0.2232]),
    #transforms.Normalize(mean=[0.4914, 0.4822, 0.4465],std = [0.1953, 0.1925, 0.1942]),
    #
])
transform_train_mask = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((32, 32),interpolation=InterpolationMode.NEAREST_EXACT),
])


transform_val = transforms.Compose([
    transforms.ToTensor(),
    transforms.RandomResizedCrop(size =(224,224), scale=(0.2, 1)),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.RandomHorizontalFlip(p=0.5),
    #transforms.Normalize(mean=[0.4823, 0.4823, 0.4823],std = [0.2232, 0.2232, 0.2232]),
    #transforms.Normalize(mean=[0.4914, 0.4822, 0.4465],std = [0.1953, 0.1925, 0.1942])
])
transform_val_mask = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((224, 224), interpolation=InterpolationMode.NEAREST_EXACT),
])


In [ ]:
#@title Dataset
# print("Path to dataset files:", path)
path = ""

#Source: https://www.kaggle.com/code/akshar1812/lung-segmentation
class LungSegmentationDataset(Dataset):
    def __init__(self, df=None, base_dir=None, resize=None, transform=None, target_transform=None, both_transform=None):
        self.data = df
        self.base_dir = Path(base_dir)
        self.resize = resize
        self.transform = transform
        self.target_transform = target_transform
        self.both_transform=both_transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_relative_path = self.data.iloc[idx, 0]#0
        label = self.data.iloc[idx, 1]#1
        img_path = self.base_dir / img_relative_path
        # Load image and mask using OpenCV
        image = cv2.imread(str(img_path), cv2.IMREAD_GRAYSCALE)
        if image is None:
            raise FileNotFoundError(f"Image not found at path: {img_path}")
        #print(image.shape)
        image = np.stack([image] * 3, axis=-1)#fjern
        #print(image.shape)
        if self.transform:
            image = self.transform(image)
        #print(torch.unique(mask))
        #mask = mask.long()
        #print("image")
        return image, label

#
#image = torch.from_numpy(image).permute(2,0,1).float()
#num_classes
df_data = pd.read_csv("trainoct2.csv")

train_df, val_df = train_test_split(df_data, test_size=0.2, random_state=42) #THE ANSWER TO THE QUESTION




In [ ]:
#@title Training Parameters
models = [] #Dino Frozen, DinoLora
rs =  [0,10,225]#255, 0, 1, 3, 5, 10, 20] #255 being a dummy class for the full optimization no LORA
num_classes = 4
learning_rate = 1e-2
num_epochs = 10
batch_size=32
num_layers = 1
input_dim = 384*num_layers
weight_decay = 1e-3
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_dataset = LungSegmentationDataset(df=train_df, base_dir=path, resize=448,
                                                    transform=transform_train, target_transform=transform_train_mask)

val_dataset = LungSegmentationDataset(df=val_df, base_dir=path, resize=448,
                                      transform=transform_val, target_transform=transform_val_mask)


train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
#@title LinearHead

class LinearHead(nn.Module):
    def __init__(self, dino, r ,in_features=1920, out_features=4, bias=True):
        super(LinearHead, self).__init__()
        self.linear = nn.Linear(in_features, out_features, bias)
        if r == 0:
            for param in dino.parameters():
                param.requires_grad = False
            self.lora_dino = dino
        elif r == 255:
            self.lora_dino = dino
        else:
            self.lora_dino = LoRA_ViT(dino, r=r, alpha=8, num_classes=0)

    def forward(self, imgs, num_layers):
        feature = self.lora_dino.get_intermediate_layers(imgs, num_layers, return_class_token=True)
        feature = feature[0][0][:, 0]#####
        outputs = []
        outputs = self.linear(feature)
        #outputs = torch.stack(outputs, dim=1)
        return outputs



In [ ]:
a = cv2.imread("2OCT2017/train/NORMAL/NORMAL-9651399-4.jpeg")
print(a)

In [ ]:
#@title Training
import tqdm
from torchmetrics import StatScores
df = pd.DataFrame(columns=["r", "loss", "dice_metric", "trainloss","trainable", "best_model", "time"])
for r in rs:
    dinov2_vits14 = torch.hub.load('facebookresearch/dinov2', 'dinov2_vits14')
    model = LinearHead(dinov2_vits14, r, input_dim, num_classes).to(device)
    criterion = nn.CrossEntropyLoss()
    #criterion = nn.CrossEntropyLoss(weight =torch.tensor([0.25783639143730885, 0.7421636085626911]).to(device))
    #criterion = CrossEntropyLoss()
    optimizer = optim.AdamW(filter(lambda a : a.requires_grad, model.parameters()), lr=learning_rate)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, num_epochs*len(train_loader.dataset), eta_min=0.0001)
    Ss = StatScores(task="multiclass", num_classes=4 ,average='micro').to(device)

    num_trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print("trainable parameters", num_trainable_params)
    iteration = {"r": r, "loss": [], "dice_metric": [], "trainloss": [],"trainable":num_trainable_params , "best_model": None, "time":None}

    dice_metric_max = 0
    start = time.time()
    val_metric_max =0
    print(num_epochs)
    itpertrain = len(train_loader)
    for epoch in tqdm.tqdm(range(num_epochs)):

        print("Current Epoch: ", epoch)
        model.train()
        running_lorunning_loss = 0
        losscomp = 0
        for i, (imgs, labels) in enumerate(train_loader):
            imgs = imgs.to(device)
            labels = labels.to(device)
            outputs = model(imgs, num_layers)
            loss = criterion(outputs, labels)
            losscomp += loss.item()
            outputs = torch.argmax(outputs, dim=1).to(device)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            scheduler.step()

        model.eval()
        val_loss = 0.0
        val_ss = Ss([0,1,2,3], [0,1,2,3]).to(device)
        with torch.no_grad():
            for i, (imgs, label) in enumerate(val_loader):
                imgs = imgs.to(device)
                label = label.to(device)
                outputs = model(imgs, num_layers)
                loss = criterion(outputs, label)
                val_loss += loss.item()
                preds = torch.argmax(outputs, dim=1).to(device)
                val_ss += Ss(preds, label).to(device)
                #print(val_ss)
                #patch_labels = patch_labels.view(-1, int(np.sqrt(feature[0][0].shape[1])), int(np.sqrt(feature[0][0].shape[1])))
            avg_val_loss = val_loss / len(val_loader)
            print("Average Loss: ", avg_val_loss)
            if avg_val_loss > val_metric_max:
                val_metric_max = avg_val_loss
                iteration["best_model"] = model
            iteration["loss"].append(avg_val_loss)
            iteration["dice_metric"].append(val_ss)#ss
            iteration["trainloss"].append(losscomp/itpertrain)
            print(val_ss)
    end = time.time()
    length = end - start
    iteration["time"] = length
    iteration_df = pd.DataFrame([iteration])
    df = pd.concat([df, iteration_df], ignore_index=True)


In [ ]:
iteration["best_model"]

In [ ]:
#r0 = df[df["r"]==0]185090 for 10
#torch.save(r0["best_model"][0].state_dict(), "r01.pt")
r0 = df[df["r"]==255]
torch.save(iteration["best_model"], "r0255.pt")
#r0 = df[df["r"]==255]
#torch.save(r0["best_model"][0].state_dict(), "r0255.pt")

In [ ]:
# for epoch in range(2):

#         print("Current Epoch: ", epoch)
#         model.train()
#         running_lorunning_lossss = 0.0
#         for i, (imgs, labels) in enumerate(train_loader):
#             imgs = imgs.to(device)
#             #print(imgs.shape)
#             labels = labels.to(device)
#             outputs = model(imgs, num_layers)
#             loss = criterion(outputs, labels)
#             outputs = torch.argmax(outputs, dim=1).to(device)
#             #print(Ss(outputs, labels))
#             #print(Ss(outputs, labels))
#             optimizer.zero_grad()
#             loss.backward()
#             optimizer.step()
#             scheduler.step()

#         model.eval()
#         val_loss = 0.0
#         val_ss = torch.tensor([0,0,0,0,0]).to(device)
#         with torch.no_grad():
#             for i, (imgs, label) in enumerate(val_loader):
#                 imgs = imgs.to(device)
#                 label = label.to(device)

#                 outputs = model(imgs, num_layers)

#                 loss = criterion(outputs, label)
#                 val_loss += loss.item()


#                 preds = torch.argmax(outputs, dim=1).to(device)
#                 val_ss += Ss(preds, label).to(device)
#                 #print(val_ss)
#                 #patch_labels = patch_labels.view(-1, int(np.sqrt(feature[0][0].shape[1])), int(np.sqrt(feature[0][0].shape[1])))
#             avg_val_loss = val_loss / len(val_loader)
#             print("Average Loss: ", avg_val_loss)
#             if avg_val_loss > val_metric_max:
#                 val_metric_max = avg_val_loss
#                 iteration["best_model"] = model
#             iteration["loss"].append(avg_val_loss)
#             iteration["dice_metric"].append(val_ss)#ss
#             print(val_ss)

sing cache found in /root/.cache/torch/hub/facebookresearch_dinov2_main
r
trainable parameters 56066
Current Epoch:  0
Average Loss:  0.5955867114843745
tensor([761, 286,   0,   0, 761], device='cuda:0')
Current Epoch:  1
Average Loss:  0.6074274760304075
tensor([761, 286,   0,   0, 761], device='cuda:0')
Current Epoch:  2
Average Loss:  0.4687202807628747
tensor([725, 200,  86,  36, 761], device='cuda:0')
Current Epoch:  3
Average Loss:  0.451808985830708
tensor([755, 241,  45,   6, 761], device='cuda:0')
Current Epoch:  4
Average Loss:  0.385383073127631
tensor([673,  74, 212,  88, 761], device='cuda:0')
Current Epoch:  5
Average Loss:  0.3380863786195264
tensor([649,  30, 256, 112, 761], device='cuda:0')
Current Epoch:  6
Average Loss:  0.3001688461412083
tensor([700,  74, 212,  61, 761], device='cuda:0')
Current Epoch:  7
Average Loss:  0.4694155455764496
tensor([749, 200,  86,  12, 761], device='cuda:0')
Current Epoch:  8
Average Loss:  0.3109981073348811
tensor([662,  25, 261,  99, 761], device='cuda:0')
Current Epoch:  9
Average Loss:  0.23698267735766643
tensor([700,  36, 250,  61, 761], device='cuda:0')

Using cache found in /root/.cache/torch/hub/facebookresearch_dinov2_main

trainable parameters 22057346
Current Epoch:  0


In [ ]:
# #@title Training
# from torchmetrics import StatScores
# df = pd.DataFrame(columns=["r", "loss", "dice_metric", "trainable", "best_model", "time"])
# for r in rs:
#     dinov2_vits14 = torch.hub.load('facebookresearch/dinov2', 'dinov2_vits14')
#     model = LinearHead(dinov2_vits14, r, input_dim, num_classes).to(device)
#     criterion = nn.CrossEntropyLoss()
#     #criterion = CrossEntropyLoss()
#     optimizer = optim.AdamW(filter(lambda a : a.requires_grad, model.parameters()), lr=learning_rate)
#     scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, num_epochs*len(train_loader.dataset), eta_min=0.0001)
#     Ss = StatScores(task="Binary", average='micro').to(device)

#     num_trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
#     print("trainable parameters", num_trainable_params)
#     iteration = {"r": r, "loss": [], "dice_metric": [], "trainable":num_trainable_params , "best_model": None, "time":None}

#     dice_metric_max = 0
#     start = time.time()
#     val_metric_max =0
#     for epoch in range(num_epochs):

#         print("Current Epoch: ", epoch)
#         model.train()
#         running_lorunning_lossss = 0.0
#         for i, (imgs, labels) in enumerate(train_loader):
#             imgs = imgs.to(device)
#             #print(imgs.shape)
#             labels = labels.to(device)
#             outputs = model(imgs, num_layers)
#             loss = criterion(outputs, labels)
#             outputs = torch.argmax(outputs, dim=1).to(device)
#             #print(Ss(outputs, labels))
#             #print(Ss(outputs, labels))
#             optimizer.zero_grad()
#             loss.backward()
#             optimizer.step()
#             scheduler.step()

#         model.eval()
#         val_loss = 0.0
#         val_ss = torch.tensor([0,0,0,0,0]).to(device)
#         with torch.no_grad():
#             for i, (imgs, label) in enumerate(val_loader):
#                 imgs = imgs.to(device)
#                 label = label.to(device)

#                 outputs = model(imgs, num_layers)

#                 loss = criterion(outputs, label)
#                 val_loss += loss.item()


#                 preds = torch.argmax(outputs, dim=1).to(device)
#                 val_ss += Ss(preds, label).to(device)
#                 #print(val_ss)
#                 #patch_labels = patch_labels.view(-1, int(np.sqrt(feature[0][0].shape[1])), int(np.sqrt(feature[0][0].shape[1])))
#             avg_val_loss = val_loss / len(val_loader)
#             print("Average Loss: ", avg_val_loss)
#             if avg_val_loss > val_metric_max:
#                 val_metric_max = avg_val_loss
#                 iteration["best_model"] = model
#             iteration["loss"].append(avg_val_loss)
#             iteration["dice_metric"].append(val_ss)#ss
#             print(val_ss)
#     end = time.time()
#     length = end - start
#     iteration["time"] = length
#     iteration_df = pd.DataFrame([iteration])
#     df = pd.concat([df, iteration_df], ignore_index=True)


In [ ]:
df

In [ ]:
# #@title Time
# from torchmetrics import StatScores
# df = pd.DataFrame(columns=["r", "time"])
# rs = [0]
# for r in rs:
#     dinov2_vits14 = torch.hub.load('facebookresearch/dinov2', 'dinov2_vits14')
#     model = LinearHead(dinov2_vits14, r, input_dim, num_classes).to(device)
#     criterion = nn.CrossEntropyLoss()
#     #criterion = CrossEntropyLoss()
#     optimizer = optim.AdamW(filter(lambda a : a.requires_grad, model.parameters()), lr=learning_rate)
#     scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, num_epochs*len(train_loader.dataset), eta_min=0.0001)
#     Ss = StatScores(task="Binary", average='micro').to(device)

#     num_trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
#     print("trainable parameters", num_trainable_params)
#     iteration = {"r": r,"time":None}

#     dice_metric_max = 0
#     start = time.time()
#     val_metric_max =0
#     for epoch in range(num_epochs):

#         print("Current Epoch: ", epoch)
#         model.train()
#         running_lorunning_lossss = 0.0
#         for i, (imgs, labels) in enumerate(train_loader):
#             imgs = imgs.to(device)
#             #print(imgs.shape)
#             labels = labels.to(device)
#             outputs = model(imgs, num_layers)
#             loss = criterion(outputs, labels)
#             outputs = torch.argmax(outputs, dim=1).to(device)
#             #print(Ss(outputs, labels))
#             #print(Ss(outputs, labels))
#             optimizer.zero_grad()
#             loss.backward()
#             optimizer.step()
#             scheduler.step()
#     end = time.time()
#     length = end - start
#     iteration["r"] = r
#     iteration["time"] = length
#     iteration_df = pd.DataFrame([iteration])
#     df = pd.concat([df, iteration_df], ignore_index=True)


In [ ]:
df

In [ ]:
#[tp, fp, tn, fn, sup]
#idx r 	time
#0 	3 	3289.828014
#1 	255 3470.051729
#2  0


In [ ]:
time.sleep(10)


In [ ]:
start = time.time()
time.sleep(10)
end = time.time()
print(end-start)